In [ ]:
!pip install spotipy
!pip install pyvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 3.2 MB/s 
     |████████████████████████████████| 140 kB 29.1 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 592 kB 1.9 MB/s 
     |████████████████████████████████| 1.6 MB 35.6 MB/s 
  Created wheel for pyvis: filename=pyvis-0.3.0-py3-none-any.whl size=600250 sha256=48b848fc0eb480ef12fe12aa9a28371d4dffb9845064a27e8f1a9aacc5cdef80
  Stored in directory: /root/.cache/pip/wheel

In [ ]:
import pandas as pd
import numpy as np
import random
import altair as alt
import requests
import inspect
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pyvis
from pyvis import network as net
from itertools import combinations
from community import community_louvain
from copy import deepcopy
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# storing the credentials:
CLIENT_ID = "116bae2a86fd4737862816c5f45d4c36"
CLIENT_SECRET = "4f4a732d83d04cfa94acc26d2b77169f"
my_username = "sx47r9lq4dwrjx1r0ct9f9m09"

# instantiating the client
# source: Max Hilsdorf (https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6)
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Bailey's Section     

Questions:
* Are there correlations betwen popularity and audio features?
* Construct a simple linear regression with popularity as the response variable, and quantitative variables within all the audio features. Train and test the performance of the regression with R^2.
* Within the Top40 subsets, what are the top 3 picked songs and plot a graph of related artists within 1 generation for each artist.
* Pick 3 users and examine if there are some patterns in danceability and speechiness.

In [ ]:
all_play_list = ["6Er5rSv7eawUDyK2gzSa47", "3wfcrfxMNAQZRnfjvSnoNv", "4sWTtloXKghMS1cz48f2qI","2Jts9QKHfyaGi0oeVFSeN0","1g44fvkwYsjasjPcy0HGOC",
                 "0tjQYEAydt0REulno4PYiC","5YS9zh4bCOpJqhenIJfFyw","4NC377bgf15Vwymy1yMR9A","0opioCiJiL23i467NPjq5O","4OsLXBh1nRf9638lyC0wuI",
                 "4kiakE4MVsxanqJHdTJwDU","6rao6zxPX8rBRZYQBtg0lh","16rZpzLucvgWLOEULEyKBM","7fqBcrxNGMjbwTAeCBTzL6","1jPGGXaawH9yUzTZBei5Hi",
                 "6r9PUfQYGPaBSnYve8cUHf","6ThSPlQWB8mdyyJPqTLxyd","5szvg6CvBqEmY2axuwweNT","5zUcUebsycpS23ffo3Av5C","45uK9eH3RujsqlSRVMW98K",
                 "6JaCaTSXvLCEXlPXswEIH8","63i6KMziOwU2yfCU7FJAfe","1U3moBdv9hDIy4FVSF0NDl","0dWk3ugBnoaztsCzY2WR4R","4hL4BxaNoBHiPfoDgQeqUA",
                 "5cdB8qvSa0V42uHE54W4X1","1xOIzbTTAgirN5PdfWfqnz","05TJaXT9IQeLnB0sNStFlo","5taHWErP8pC1xhtSWWPYLE","3k606c4g6XPJFPupzmwSNJ",
                 "53qBETrCJQ6EFMro3RFeVN","5MPTim2q6agXKcYyEGeJjc","6t1GwwI6JvES6I3PWrmgIr","1ECGflprVeWBWgJKqh2We5","7hIYT6eLgtTgFeEwaGqY6l",
                 "7I076Vit9Fd65C9fVNlPy0","4xV24s6m1s6mPrilnsBqBa","5QxWG2oKSylpTg9qS5rPRr","5NuMif6wvlJeJv4dMuX5eh"]
top_40 = ["1NppEwvZhkjeG3ZTYoOwVM","2hfOGugGPsjfPTYKlZojom","6QWlhkFUKDxhW91gfMisNl","1d4mKuQzM62pmSqrQKU9EX","07UNN5sIx1dYmAOmLPri3B",
          "3cfp7fWCHVXPl6JiBnmkLQ","7E5YHVj7yA5SVthGIAJ67g","5NuMif6wvlJeJv4dMuX5eh","5QxWG2oKSylpTg9qS5rPRr","4xV24s6m1s6mPrilnsBqBa","1kaDWd90UWUByF4Tu2UPVx",
          "3DmSmomqxsSehMhccpPbWP","3Nd7XuKMH3RjQ6nF7n2kcv","5LVjFO57XKpRNV5vzzDmP5","4AITV0hEKFEjLcy5dJBrtC","0QZa8PuiIKpSRDXFcbc2y2"]

The above cells contains two list. "all_play_list" is a list containing mostly the other lists, and "top_40" contains all top_40 playlists.

In [ ]:
def get_audio_features_df(playlist):
    
    # Create an empty dataframe
    playlist_features_list = ["artist", "album", "track_name", "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    for track in playlist["items"]:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["popularity"] = track["track"]["popularity"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the DataFrames
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [ ]:
playlist_df_list = []
for playlist in all_play_list:
  temp_playlist_df = pd.DataFrame(sp.playlist_items(playlist))
  temp_playlist_audio = get_audio_features_df(temp_playlist_df)
  temp_playlist_audio["playlist_name"] = sp.playlist(playlist)["name"]
  playlist_df_list.append(temp_playlist_audio)
    
# Concatenating the Audio Features DataFrames
playlists_df = pd.concat(playlist_df_list)
playlists_df.to_csv("full.csv")

df_list = []
for playlist in top_40:
  temp_playlist_df = pd.DataFrame(sp.playlist_items(playlist))
  temp_playlist_audio = get_audio_features_df(temp_playlist_df)
  temp_playlist_audio["playlist_name"] = sp.playlist(playlist)["name"]
  df_list.append(temp_playlist_audio)
    
# Concatenating the Audio Features DataFrames
playlists = pd.concat(df_list)
playlists.to_csv("top40.csv")


The above cell gets all the audio features and popularity for all the tracks in the "all_play_list" storing some playlists and concatenate them into one dataframe. Finally, it will save the dataframe to a csv to make the data reusable and save some time by not calling spotify API every time. Information on the top_40 playlists are also saved into a csv. 

In [ ]:
data = pd.read_csv("full.csv")
plt.figure(figsize=(16,16))
sns.heatmap(data.corr(), annot=True, cmap="Greens")
plt.show()

In the above cells, I used almost all playlists and concatenate all the songs' audio features into one single dataframe. Then, I use a heatmap to visualize the correlation. As we can see, most of the features are not highly correlated linearly. "loudness" is highly correlated with "energy". Looking at the popularity row (the last row), we can see that loudness has the highest correlation with popularity. 

In [ ]:
quantitative_variables = ["danceability","energy","loudness","speechiness",
                          "instrumentalness","liveness","valence","tempo","duration_ms"]
X = data[quantitative_variables].to_numpy()
y = data["popularity"].to_numpy()
x_train, x_test, y_train,  y_test = train_test_split(X,y, train_size=0.7, shuffle=True, random_state=42)

In the above cell, I store all the quantitative variables within the audio features into X (the feature matrix) and the response variable popularity into y. Then, I split the X,y into training and testing sets. The training set will be used to train the linear regression model, and the testing set will only be used to test the performance of the regression using R^2. 

In [ ]:
model = LinearRegression()
model.fit(x_train,y_train)
weights = model.coef_
intercept = model.intercept_
test_predictions = model.predict(x_test)
print(f"The mean sqaured error is: {mean_squared_error(y_test,test_predictions)}")
print(f"R^2 is: {model.score(x_test,y_test)}")

The mean sqaured error is: 384.7595631156008
R^2 is: 0.1657521435660012


In the above cell, I fitted a linear regression between the audio features and popularity. We can see that R^2 is really low which means that only 16.68% variation is the audio features are explained by their linear relationship with popularity. In other words, the current linear regression is not a good model for predicting the popularity. We can do some transformations such as box-cox transformation to improve the performance our model, but transformation of linear regression itself is a huge project, so I am not going to details here. Another problem that may contribute the failure of the model is that there could be other predictors that heavily influence popularity but are not included in our model. Our previous heatmap of linear correlation suggests that linear regression without transformation may not work since audio features don't have high correlation with popularity.

In [ ]:
top40_data = pd.read_csv("top40.csv")
print(top40_data[["artist","track_name","popularity"]])
print(top40_data['track_name'].value_counts())

                 artist                  track_name  popularity
0         Frank Sinatra      Strangers In The Night        70.0
1       The Temptations                     My Girl        80.0
2       Aretha Franklin       I Say a Little Prayer        59.0
3       Ella Fitzgerald  Dream A Little Dream Of Me        48.0
4          Janis Joplin                       Maybe        59.0
..                  ...                         ...         ...
156  Jefferson Airplane                White Rabbit        72.0
157        Otis Redding     Try a Little Tenderness        63.0
158         Nina Simone                   Sinnerman        63.0
159     Louis Armstrong      What A Wonderful World        69.0
160     Ella Fitzgerald  Dream A Little Dream Of Me        48.0

[161 rows x 3 columns]
What A Wonderful World                                                          9
I Say a Little Prayer                                                           9
My Girl                                     

In the above cell, I computed how many times each track is selected as part of a top_40 playlist. And the top5 tracks are: What A Wonderful World, I Say a Little Prayer, My Girl.

In [ ]:
def get_related_artists(pd1,pd2,pd3):
  artists=[]
  result = []
  names1 = pd1["name"].to_numpy()  
  names2 = pd2["name"].to_numpy()
  names3 = pd3["name"].to_numpy()
  for name in names1:
    artists.append(name)
  
  for name in names2:
    if name in artists:
      result.append(name)
  
  for name in names3:
    if name in artists:
      result.append(name)
  
  for name in names2:
    if name in names3 and name not in artists:
      result.append(name)
  
  if (len(result)==0):
    return "No related artists found!"
  return result

The above function takes three parameters which are 3 dataframes containing the related artist names. This function will return a list of related artists for the 3 dataframes.

In [ ]:
louis_armstrong = "19eLuQmk9aCobbVDHc6eek"
popularity1=69.0
name1="Louis Armstrong"
aretha_franklin ="7nwUJBm0HE4ZxD3f5cy5ok"
popularity2=59.0
name2 = "Aretha Franklin"
the_temptations ="3RwQ26hR2tJtA8F9p2n7jG"
popularity3=80
name3 = "The Temptations"

armstrong_related = pd.DataFrame(sp.artist_related_artists(louis_armstrong)["artists"])
franklin_related = pd.DataFrame(sp.artist_related_artists(aretha_franklin)["artists"])
temptations_related = pd.DataFrame(sp.artist_related_artists(the_temptations)["artists"])
related_artist = get_related_artists(armstrong_related,franklin_related,temptations_related)
print(related_artist)

['Gladys Knight & The Pips', 'Jackie Wilson', 'The Supremes', 'Smokey Robinson & The Miracles', 'Four Tops', 'Martha Reeves & The Vandellas', 'The Spinners', 'Harold Melvin & The Blue Notes']


From the above output, we can see that Louis Armstrong, Aretha Franklin, and The Temptations are actually connected through other artists within their generation.

In [ ]:
def add_related_artists(starting_artist_name, starting_artist_id, existing_graph, limit, order_group=None):
    # get artists related to the current artist
    current_artist_related = pd.DataFrame(sp.artist_related_artists(starting_artist_id)["artists"])
    # loop through the related artists, add nodes and edges
    for i in range(limit):
        # check if node already exists
        if current_artist_related.loc[i]["name"] not in existing_graph.get_nodes():
            if order_group:
                existing_graph.add_node(current_artist_related.loc[i]["name"], value=int(current_artist_related.loc[i]["popularity"]), group=order_group)
            else:
                existing_graph.add_node(current_artist_related.loc[i]["name"], value=int(current_artist_related.loc[i]["popularity"]), group=(i + 1))
        # add edge

        existing_graph.add_edge(starting_artist_name, current_artist_related.loc[i]["name"])

In [ ]:
limit=10
armstrong_related = pd.DataFrame(sp.artist_related_artists(louis_armstrong)["artists"]).loc[0:(limit-1)]
franklin_related = pd.DataFrame(sp.artist_related_artists(aretha_franklin)["artists"]).loc[0:(limit-1)]
temptations_related = pd.DataFrame(sp.artist_related_artists(the_temptations)["artists"]).loc[0:(limit-1)]

# armstrong network
artist_network1 = net.Network(notebook=True)
artist_network1.add_node(name1, value=popularity1, color="#fffff", group=0)
add_related_artists(name1, louis_armstrong, artist_network1, limit)
for i in range(limit):
    add_related_artists(armstrong_related.loc[i]["name"], armstrong_related.loc[i]["id"], artist_network1, limit, (i+1))
artist_network1.show("armstrong.html")

# franklin network
artist_network2 = net.Network(notebook=True)
artist_network2.add_node(name2, value=popularity2, color="#fffff", group=0)
add_related_artists(name2, aretha_franklin, artist_network2, limit)
for i in range(limit):
    add_related_artists(franklin_related.loc[i]["name"], franklin_related.loc[i]["id"], artist_network2, limit, (i+1))
artist_network2.show("franklin.html")

# temptations network
artist_network3 = net.Network(notebook=True)
artist_network3.add_node(name3, value=popularity3, color="#fffff", group=0)

add_related_artists(name3, the_temptations, artist_network3, limit)
# Running through the once-related artists
for i in range(limit):
    add_related_artists(temptations_related.loc[i]["name"], temptations_related.loc[i]["id"], artist_network3, limit, (i+1))

# Showing the Network Graph
artist_network3.show("temptations.html")


colab doesn't show the network. To view the network, please look at colab files and download 3 html files: "armstrong.html","franklin.html","temptations.html". After downloading the files, open them in any internet browser. In the above cell, I built 3 networks for Louis Armstrong, Aretha Franklin, and the Temptations respectively. 

In [ ]:
gs_playlist_tracks = pd.DataFrame(sp.user_playlist_tracks("313pk3sai2bekdmtoefrbjmbv5ey", "6Er5rSv7eawUDyK2gzSa47"))
gs_playlist_tracks_audio_df = get_audio_features_df(gs_playlist_tracks)
gs_playlist_tracks_audio_df["Author"] = "Author1"

gs_playlist_tracks2 = pd.DataFrame(sp.user_playlist_tracks("316b7eir5onpduebous2uyrqwxue", "4sWTtloXKghMS1cz48f2qI"))
gs_playlist_tracks_audio_df2 = get_audio_features_df(gs_playlist_tracks2)
gs_playlist_tracks_audio_df2["Author"] = "Author2"

gs_playlist_tracks3 = pd.DataFrame(sp.user_playlist_tracks("4uxd1wpjsxqvlpni1ixj09ipn", "5YS9zh4bCOpJqhenIJfFyw"))
gs_playlist_tracks_audio_df3 = get_audio_features_df(gs_playlist_tracks3)
gs_playlist_tracks_audio_df3["Author"] = "Author3"

combined = pd.concat([gs_playlist_tracks_audio_df, gs_playlist_tracks_audio_df2, gs_playlist_tracks_audio_df3], ignore_index=True)

In the above cell, I picked 3 playlists from 3 users and combine them into one dataframe for the graph below.

In [ ]:
alt.Chart(combined).mark_point().encode(
    x=alt.X("track_name", sort=None),
    y='danceability',
    color="Author",
    tooltip=["artist", "track_name"]
).properties(
    width=1000
)

alt.Chart(...)

In the above cell, I plotted the danceability of all the songs from the 3 playlists, and we can see that most of the danceability lied between 0.2 and 0.6. For these 3 users, they prefer lower danceability.

In [ ]:
alt.Chart(combined).mark_point().encode(
    x=alt.X("track_name", sort=None),
    y='speechiness',
    color="Author",
    tooltip=["artist", "track_name"]
).properties(
    width=1000
)

alt.Chart(...)

In the above cell, I plotted the speechiness of all the songs from the 3 playlists, and we can see that all songs have pretty low speechiness. 

#Ben's Section

Degrees of removal measurement:
Given Artist X and Artist Y, how many generations of related artists to X does it take to find Y?

This metric is inspired by the "6 degrees of seperation" theory:
https://en.wikipedia.org/wiki/Six_degrees_of_separation


The algorithm:

Generate 5 related artists for both X and Y. Then generate 5 artists for each of these related artists and repeat this process, only adding artists that have not yet been added. Keep a running list of artists in the X tree and in the Y tree. If at any point an Artist Z is found in both lists, then calculate the shortest path between X and Y through Z. If 5 generations are run on each side before any Z is found, then return -1 (signifying no relation between X and Y)

Notes:

The space and runtime complexity can be very high, so we limit the number of generations to 5 each (2*(5^5) items to return in worst case). Spotify can only generate 20 related artists to a given artist, so the total data frame is not guaranteed to be that large (e.g. if all 20 of an artist's related artists are already in the data frame).



In [ ]:
# (BM)
bobdylan = "74ASZWbe4lXaubB36ztrGX"
arianagrande = "66CXWjxzNUsdJxJ2JdwvnR"
beatles = "3WrFJ7ztbogyGnTHbHJFl2"
lennon = "4x1nvY2FN8jxqAFA0DA02H"


def get_five_related(artist,generation, currentTree):
  related_artists = pd.DataFrame(sp.artist_related_artists(artist)["artists"])
  data = []
  i = 0
  while (len(data)<5 and i<20):
    if related_artists["id"][i] not in currentTree.values:   #### issue here
      data.append([related_artists["id"][i], generation+1,related_artists["name"][i]])
    i+=1
  return pd.DataFrame(data, columns = ["artists", "generation","name"])


def degree_of_seperation(X,Y):

  current_X_tree = pd.DataFrame([[X,0, sp.artist(X)["name"]]], columns = ["artists", "generation","name"])
  current_Y_tree = pd.DataFrame([[Y,0, sp.artist(Y)["name"]]], columns = ["artists", "generation","name"])
  
  starting_X = 0
  starting_Y = 0

  for generation in range(5):

    temp_X_tree = pd.DataFrame(columns = ["artists", "generation","name"])

    for index in range(starting_X,len(current_X_tree)):
      new_X_tree = get_five_related(current_X_tree["artists"].iloc[index],generation,current_X_tree)

      for index in range(len(new_X_tree)):
        check_artist = new_X_tree["artists"].iloc[index]
        # if there is a match
        if check_artist in current_Y_tree["artists"].values:  ##### issue here
          # return the sum of the two generation counts
          y_val = current_Y_tree[current_Y_tree["artists"]==check_artist]
          return new_X_tree["generation"].iloc[index] + y_val["generation"].iloc[0]
      temp_X_tree = pd.concat([temp_X_tree, new_X_tree])
    
    # update starting index
    starting_X = len(current_X_tree)
    # update current_X_tree with temp_X_tree
    current_X_tree = pd.concat([current_X_tree, temp_X_tree])

    temp_Y_tree = pd.DataFrame(columns = ["artists", "generation","name"])

    for index in range(starting_Y,len(current_Y_tree)):
      new_Y_tree = get_five_related(current_Y_tree["artists"].iloc[index],generation,current_Y_tree)
      
      for index in range(len(new_Y_tree)):
        check_artist = new_Y_tree["artists"].iloc[index]
        # if there is a match
        if check_artist in current_X_tree["artists"].values:    ###### issue here
          # return the sum of the two generation counts
          x_val = current_X_tree[current_X_tree["artists"]==check_artist]
          return new_Y_tree["generation"].iloc[index] + x_val["generation"].iloc[0]
      temp_Y_tree = pd.concat([temp_Y_tree, new_Y_tree])
      
    # update starting index
    starting_Y = len(current_Y_tree)
    # update current_Y_tree with temp_Y_tree
    current_Y_tree = pd.concat([current_Y_tree, temp_Y_tree])
  
  return -1

In [ ]:
# should return 1
degree_of_seperation(beatles, lennon)

1

In [ ]:
# returned 4 when I ran this
degree_of_seperation(beatles, bobdylan)


4

In [ ]:
# took a while to run, but eventually returned 10 (coincidentally, this is the max degree of seperation this algorithm can find)
degree_of_seperation(arianagrande, bobdylan)

The cells above shows the potential for using this "degree of separation" metric for analyzing playlists and comparing artists. We may find that artists for an "imagned persona" playlist are more or less removed from one another than artists for a "mood" playlist. Artists in "Genre A" may have more variation in degree of removal than artists from "Genre B."

In the cell below, I will compare average degree of seperation between artists in three playlists: 
*   "Songs to Punch Drywall To"
*   "your 20s are for losing your mind"
*   "Hallelujah Covers"

My hypothesis in this analyzation is that the average degree of seperation will be lower for the imagined persona playlist (your 20s are for losing your mind) than in the thematic playlist (Songs to Punch Drywall To). My logic here is that an imagined person is likely to listen to related artists, whereas an imagned mood might span across a wider range of artists. In addition, I am curious to see the average degree of seperation for the playlist of covers.




In [ ]:
user1 = "4uxd1wpjsxqvlpni1ixj09ipn"
rage = "0opioCiJiL23i467NPjq5O"

user2 = "313pk3sai2bekdmtoefrbjmbv5ey"
twenties = "6Er5rSv7eawUDyK2gzSa47"

user3 = "316b7eir5onpduebous2uyrqwxue"
hallelujah = "4sWTtloXKghMS1cz48f2qI"

def listofartists(playlist,username):
    pl = pd.DataFrame(sp.user_playlist_tracks(username, playlist))
    output = []
    for track in pl["items"]:
        output.append(track["track"]["album"]["artists"][0]["id"])
    return output

def average_dos(playlist,username):
  data = []
  for artist1 in listofartists(playlist,username):
    for artist2 in listofartists(playlist,username):
      if artist1 != artist2:
        data.append(degree_of_seperation(artist1,artist2))
  dataframe = pd.DataFrame(data, columns = ["degree of seperation"]) # this data frame could be useful for other types of analysis later on
  if -1 in data:
    return "Error: could not find the degree of seperation for all artists"  # unfortunately, I am not sure what to do in the case that no match is found
  else:
    return sum(data)/len(data)




In [ ]:
# would expect this to be relatively high
average_dos(rage,user1)

KeyboardInterrupt: ignored

In [ ]:
# would expect this to be relatively low
average_dos(twenties,user2)

In [ ]:
# interested to see how this compares
average_dos(hallelujah,user3)

ConnectionError: ignored